In [1]:
import numpy as np

In [41]:
n=7
k=4
matriz_P = np.array([[1,1,0],[0,1,1],[1,1,1],[1,0,1]])
matriz_Ik= np.identity(k)
matriz_G = np.concatenate((matriz_P.T,matriz_Ik.T),axis=0).T
matriz_H = np.concatenate((matriz_Ik[:-1].T[:-1],matriz_P),axis=0).T
matriz_Ht = matriz_H.T

In [50]:
## calculo de patrones de error
patron_error = np.array(["{0:07b}".format(x) for x in range(128)])
cantidad_error = np.array([x.count('1') for x in patron_error])

In [63]:
type(patron_error[0].encode("utf_8"))

bytes

In [65]:
matriz_Ht[0]

array([1., 0., 0.])